In [1]:
# Pipeline 2 description
# Fit a beta distribution to all the tissue columns w/o the nan values in the training set.
# Use the beta distributions conditioned on the location to fill the NaN values.
# Once you have finished filling in the NaNs, your training set will be complete
# To fill in the values in the test set, take the test vector's values, locations and find the
# top x distributions that the vector is most likely to come from.
# Use these top x as the training set for a regressor that will be used to predict the missing values in the training
# set.

In [54]:
# load imports
import math
import numpy as np 
from IPython.display import Image  
import pandas
import os
from scipy.stats import beta
# Enter your directory here
os.chdir('/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/methylation_imputation-master')

In [3]:
# Load the data
train_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_train.bed', sep='\t', header=None)
sample_p_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_sample_partial.bed', sep='\t', header=None)
sample_f_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_sample_full.bed', sep='\t', header=None)

In [56]:
# fit the distributions
# first trial with just a single tissue
tissue_sample = train_bed[4]
tissue_sample_list = tissue_sample
# remove NaNs
tissue_sample_no_nans = [x for x in tissue_sample_list if ~np.isnan(x)]
a, b, loc, scale = beta.fit(tissue_sample_no_nans)

In [57]:
# build the beta for the test vector
# use the beta to generate the values for the missing values.
beta_gen = np.random.beta
tissue_sample_nan_rep = [x if ~np.isnan(x) else beta_gen(a, b) for x in tissue_sample_list]

In [64]:
# now time to do it with the entire set!
def fill_col(col):
    col_sample_list = col
    col_sample_no_nans = [x for x in col_sample_list if ~np.isnan(x)]
    a, b, loc, scale = beta.fit(col_sample_no_nans)
    beta_gen = np.random.beta
    col_sample_nan_rep = [x if ~np.isnan(x) else beta_gen(a, b) for x in col_sample_list]
    return col_sample_nan_rep

table1 = train_bed[list(range(4,37))]
table2 = []
for i in range(4, 37):
    table2.append(fill_col(table1[i]))

table2 = pandas.DataFrame(table2)

In [68]:
# Now create a list of (a,b) pairs so you can figure out which distribution is the most likely 
# for your similarity metric
# table2 has all the NaN values filled in in the training set.
ab_tuples = []
def get_ab(col):
    a, b, loc, scale = beta.fit(col)
    return a,b

for i in range(4,37):
    ab_tuples.append(get_ab(table2[i]))

print len(ab_tuples)

33


In [83]:
# once we have the list of ab tuples, we can use them to calculate the probability
# that the test vector came from any one of these distributions
test_vector_nans = sample_p_bed[4] # 4th column has the data, others are informatory
test_vector_no_nans = [x for x in test_vector_nans if ~np.isnan(x)]
similarity_scores = []
i = 0
for a, b in ab_tuples:
    running_product = 0
    for meth in test_vector_no_nans:
        assert (0 < meth < 1)
        probability = beta.logpdf(meth, a, b)
        running_product = running_product + probability
    similarity_scores.append((running_product, i))
    i = i + 1

# sort the scores by the first tuple value
sorted_similarity = sorted(similarity_scores, key=lambda tup: tup[0])

[(-13659490456237456.0, 19), (-468922803856.88477, 28), (-247037332581.60791, 26), (-12298065041.707794, 31), (-10978471043.584106, 20), (-8051881059.2469997, 32), (-32805912.004562154, 7), (-4406257.3454944938, 27), (-317697.32312980184, 21), (-95469.340965576077, 18), (-49589.869748645382, 13), (-14437.079474929746, 16), (-7156.5482661051547, 0), (-6441.9824834351766, 22), (-4167.9483290805319, 23), (-4035.3804815743883, 25), (-2193.2234922808666, 17), (-1536.2867341735198, 24), (-92.968290898778704, 29), (384.45164212161984, 9), (618.98387205880874, 2), (1220.3286454002207, 4), (1383.3589021019866, 30), (1604.5434066759576, 1), (1741.0272154780146, 3), (1763.2054999725726, 6), (1826.2384759577812, 11), (1914.6054993076173, 15), (2054.0610890510643, 10), (2267.9794906005231, 8), (2397.0167268304494, 5), (2420.2267945418512, 14), (2432.5848001531817, 12)]


In [ ]:
# take the top x number of these vectors, and use their values to create a regression.
top_x = 3

